In [1]:
from rdflib import Graph, SKOS, Literal, URIRef, RDF
import pandas as pd
import utils
import pickle

In [2]:
rdf_all_graphs, label_uri_dict, uri_label_dict = utils.load_resources()


In [3]:
PREFIXES = """
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX emmo: <http://emmo.info/emmo#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX echem: <http://emmo.info/electrochemistry#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dc: <http://purl.org/dc/elements/1.1/> 
PREFIX battery: <http://emmo.info/battery#> 
PREFIX schema:<https://schema.org/>
"""

orcid = "https://orcid.org/0000-0003-2954-1233"

query2 = PREFIXES  + f"""
SELECT ?givenName ?familyName ?email

WHERE {{
    <{orcid}> schema:familyName ?familyName;
            schema:givenName ?givenName;
            schema:email ?email.
}}
"""
qres = rdf_all_graphs.query(query2)
list(qres)

[(rdflib.term.Literal('Eibar'),
  rdflib.term.Literal('Flores'),
  rdflib.term.Literal('eibar.flores.cedeno@sintef.no'))]

In [4]:
utils.pickle_resources()

In [8]:
utils.query_all_schemas(rdf_all_graphs, uri_label_dict, schema_type="productionDate")

['2023-05-02']

In [4]:
utils.query_all_materials(rdf_all_graphs, uri_label_dict)

{'LithiumNickelManganeseCobaltOxide': 'http://emmo.info/emmo#EMMO_4950a41c_ae32_4ce9_9db9_0076bbcac18f',
 'CarboxymethylCellulose': 'http://emmo.info/emmo#EMMO_5805174a_71da_4263_b4d4_c2868cceeef7',
 'Graphite': 'http://emmo.info/emmo#EMMO_5dd27e7d_c38f_4a81_8d09_d81463806ad7',
 'PolyvinylFluoride': 'http://emmo.info/emmo#EMMO_6c03b6b7_a50e_4575_9708_0d4f18a3c358'}

In [16]:
PREFIXES = """
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX emmo: <http://emmo.info/emmo#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX echem: <http://emmo.info/electrochemistry#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dc: <http://purl.org/dc/elements/1.1/> 
PREFIX battery: <http://emmo.info/battery#> 
PREFIX schema:<https://schema.org/>
"""


def query_cell_by_material(rdf_graph:Graph, label_uri_dict:dict, uri_label_dict:dict, material_label:str)->pd.DataFrame:

    material_iri = URIRef(label_uri_dict[material_label])
    print(material_iri)

    query_by_material = PREFIXES  + """
    SELECT ?instance ?name ?creator ?date
    WHERE {{
    ?instance rdf:type battery:battery_68ed592a_7924_45d0_a108_94d6275d57f0;
              schema:creator ?creator;
              schema:name ?name;
              schema:productionDate ?date;
              (echem:electrochemistry_8e9cf965_9f92_46e8_b678_b50410ce3616|echem:electrochemistry_5d299271_3f68_494f_ab96_3db9acdd3138) ?something.

    ?something (emmo:EMMO_dba27ca1_33c9_4443_a912_1519ce4c39ec)+ ?material.
    ?material rdf:type <%s>.

    }}
    """% material_iri    

    results_dict = {"CellID":[], "Cell name":[], "Creator":[], "Date":[], "Material":[]}

    qres = rdf_graph.query(query_by_material)
    for hit in qres:
        string_rep = [str(rdf_obj) for rdf_obj in hit]
        human_rep = [uri_label_dict[string] if string in uri_label_dict else string for string in string_rep]
        results_dict["CellID"].append(human_rep[0])
        results_dict["Cell name"].append(human_rep[1])
        results_dict["Creator"].append(human_rep[2])
        results_dict["Date"].append(human_rep[3])
        results_dict["Material"].append(material_label)
    
    return pd.DataFrame(results_dict)



results_df = query_cell_by_material(rdf_all_graphs, label_uri_dict, uri_label_dict, material_label="Graphite")
results_df

http://emmo.info/emmo#EMMO_5dd27e7d_c38f_4a81_8d09_d81463806ad7


,CellID,Cell name,Creator,Date,Material
0,http://rdf.batterymodel.com/96a8c962-cf63-40bf...,NMC Graphice cell with CMC,https://orcid.org/0000-0002-8758-6109,2023-05-02,Graphite


In [34]:
material_iri = URIRef(label_uri_dict["Graphite"])
material_iri 

rdflib.term.URIRef('http://emmo.info/emmo#EMMO_5dd27e7d_c38f_4a81_8d09_d81463806ad7')